In [ ]:
# 3_interpretability.ipynb
import joblib
import pandas as pd
import shap
import matplotlib.pyplot as plt
from src.preprocessing import build_preprocessing_pipeline

In [ ]:
best = joblib.load('models/best_model.joblib')
# best is a pipeline: pre + clf
# obtener X_train sample para explicar
df = pd.read_csv('data/processed/heart_raw_processed.csv')
X = df.drop(columns=['target'])
y = df['target']

In [ ]:
# Tomamos un subset para speed
X_sample = X.sample(n=500, random_state=42)

In [ ]:
# Para calcular shap, necesitamos datos transformados y el modelo final (árboles)
# Si la parte predictora es XGB, podemos crear explainer sobre el step final
# extraer pipeline componentes
pre = best.named_steps['pre']
clf = best.named_steps['clf']

X_trans = pre.transform(X_sample)

In [ ]:
# obtener feature names tras one-hot
num_cols = pre.transformers_[0][2]

In [ ]:
# obtener ohe cols
ohe = pre.transformers_[1][1].named_steps['onehot']
ohe_cols = ohe.get_feature_names_out(pre.transformers_[1][2])
feature_names = list(num_cols) + list(ohe_cols)

In [ ]:
# SHAP
explainer = shap.TreeExplainer(clf)
shap_values = explainer.shap_values(X_trans)
shap.summary_plot(shap_values, X_trans, feature_names=feature_names, show=True)
plt.savefig('reports/shap_summary.png', dpi=150)